In [19]:
import logging
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
from pprint import pprint
# Setup logger
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("BitcoinRPC")

# Bitcoin Core RPC connection details
rpc_user = "your_rpc_user"  # Replace with your RPC user
rpc_pass = "your_secure_password"  # Replace with your RPC password
rpc_host = "127.0.0.1"
rpc_port = "18443"  # Assuming regtest is running on this port

# Specify the wallet name to interact with
wallet_name = "mynewwallet"  # Replace with your wallet name if needed

# RPC client connection (this needs to be for the wallet we are using)
rpc_client = AuthServiceProxy(f'http://{rpc_user}:{rpc_pass}@{rpc_host}:{rpc_port}/wallet/{wallet_name}', timeout=120)

# Create new addresses for A, B, and C
new_address_a = rpc_client.getnewaddress("", "legacy")
address_b = rpc_client.getnewaddress("", "legacy")
address_c = rpc_client.getnewaddress("", "legacy")
logger.info(f"New Legacy address A generated: {new_address_a}")
logger.info(f"New Legacy address B generated: {address_b}")
logger.info(f"New Legacy address C generated: {address_c}")

# Send 10 BTC to Address A
txid = rpc_client.sendtoaddress(new_address_a, 10.0)
logger.info(f"Sent 10.0 BTC to address A with TXID: {txid}")

# Function to mine blocks and confirm the transaction in Regtest
def mine_blocks_to_confirm_transaction(txid, block_count=6):
    try:
        # Mine the required number of blocks to confirm the transaction
        logger.info(f"Mining {block_count} blocks to confirm the transaction {txid}...")
        
        # This will mine 6 blocks and include the transaction in them
        rpc_client.generatetoaddress(block_count, new_address_a)
        logger.info(f"{block_count} blocks mined.")

        # After mining, check the transaction confirmation count
        tx_info = rpc_client.gettransaction(txid)
        confirmations = tx_info["confirmations"]
        
        if confirmations >= 6:
            logger.info(f"Transaction {txid} has reached 6 confirmations.")
        else:
            logger.info(f"Transaction {txid} has not reached 6 confirmations yet.")
    
    except JSONRPCException as e:
        logger.error(f"Error mining blocks or checking transaction status: {e}")

# Call the function to mine 6 blocks and confirm the transaction
mine_blocks_to_confirm_transaction(txid, block_count=6)

# Check the balance of the wallet after the transaction
def check_wallet_balance():
    try:
        # Get the current balance of the wallet
        balance = rpc_client.getbalance()
        logger.info(f"Wallet balance after sending transaction: {balance} BTC")
        
        # You can also check the balance of the specific address
        address_balance = rpc_client.getreceivedbyaddress(new_address_a)
        logger.info(f"Balance of address {new_address_a}: {address_balance} BTC")
    
    except JSONRPCException as e:
        logger.error(f"Error fetching wallet balance: {e}")

# Check wallet balance after the transaction
check_wallet_balance()

# Step 3: Get UTXOs from Address A
try:
    # Get unspent transaction outputs (UTXOs) for Address A
    utxos = rpc_client.listunspent(0, 9999999, [new_address_a])
    logger.info(f"UTXOs for Address A: {utxos}")
    
    if not utxos:
        logger.error("No UTXOs found for Address A.")
        exit(1)
    
    # Select the first UTXO (this assumes a single UTXO, adjust if needed)
    selected_utxo = utxos[0]
    txid = selected_utxo["txid"]
    vout = selected_utxo["vout"]
    amount = selected_utxo["amount"]
    logger.info(f"Selected UTXO: txid={txid}, vout={vout}, amount={amount} BTC")

except JSONRPCException as e:
    logger.error(f"Error retrieving UTXOs for Address A: {e}")
    exit(1)

# Step 4: Create and sign the transaction
try:
    # Create the raw transaction input (spending UTXO from Address A)
    inputs = [{
        "txid": txid,
        "vout": vout
    }]
    
    # Create the raw transaction output (sending Bitcoin to Address B)
    outputs = {
        address_b: float(amount)-0.0001   # Send all funds minus a small fee (e.g., 0.0001 BTC)
    }
    
    # Create the raw transaction
    raw_tx = rpc_client.createrawtransaction(inputs, outputs)
    logger.info(f"Raw transaction created: {raw_tx}")
    
    # Sign the transaction
    signed_tx = rpc_client.signrawtransactionwithwallet(raw_tx)
    logger.info(f"Signed transaction hex: {signed_tx['hex']}")
    
    # Extract signed transaction hex
    signed_tx_hex = signed_tx["hex"]

except JSONRPCException as e:
    logger.error(f"Error creating or signing transaction: {e}")
    exit(1)

# Step 5: Broadcast the signed transaction to the network
try:
    # Broadcast the signed transaction
    txid_broadcast = rpc_client.sendrawtransaction(signed_tx_hex)
    logger.info(f"Broadcasted transaction with TXID: {txid_broadcast}")

except JSONRPCException as e:
    logger.error(f"Error broadcasting transaction: {e}")
    txid_broadcast = None  # If broadcasting fails, set txid_broadcast to None

# Step 6: Mine blocks to confirm the transaction (on regtest)
if txid_broadcast:
    try:
        logger.info(f"Mining 6 blocks to confirm the transaction {txid_broadcast}...")
        rpc_client.generatetoaddress(6, address_b)  # Mine 6 blocks to confirm the transaction
        logger.info("6 blocks mined.")
    except JSONRPCException as e:
        logger.error(f"Error mining blocks to confirm the transaction: {e}")
else:
    logger.error("Transaction broadcasting failed. Mining cannot proceed.")

# Step 7: Check balances after transaction
try:
    # Check balance of Address B
    balance_b = rpc_client.getreceivedbyaddress(address_b)
    logger.info(f"Balance of Address B after transaction: {balance_b} BTC")

except JSONRPCException as e:
    logger.error(f"Error checking balances: {e}")

# Step 8: Get UTXOs from Address B and create second transaction to Address C
try:
    # Get unspent transaction outputs (UTXOs) for Address B
    utxos2 = rpc_client.listunspent(0, 9999999, [address_b])
    logger.info(f"UTXOs for Address B: {utxos2}")
    
    if not utxos2:
        logger.error("No UTXOs found for Address B.")
        exit(1)
    
    # Select the first UTXO (this assumes a single UTXO, adjust if needed)
    selected_utxo2 = utxos2[0]
    txid2 = selected_utxo2["txid"]
    vout2 = selected_utxo2["vout"]
    amount2 = selected_utxo2["amount"]
    logger.info(f"Selected UTXO: txid={txid2}, vout={vout2}, amount={amount2} BTC")

except JSONRPCException as e:
    logger.error(f"Error retrieving UTXOs for Address B: {e}")
    exit(1)

# Step 9: Create and sign the transaction for Address B to Address C
try:
    # Create the raw transaction input (spending UTXO from Address B)
    inputs2 = [{
        "txid": txid2,
        "vout": vout2
    }]
    
    # Create the raw transaction output (sending Bitcoin to Address C)
    outputs2 = {
        address_c: float(amount2)-0.00001   # Send all funds minus a small fee (e.g., 0.00001 BTC)
    }
    
    # Create the raw transaction
    raw_tx2 = rpc_client.createrawtransaction(inputs2, outputs2)
    logger.info(f"Raw transaction created: {raw_tx2}")
    
    # Sign the transaction
    signed_tx2 = rpc_client.signrawtransactionwithwallet(raw_tx2)
    logger.info(f"Signed transaction hex: {signed_tx2['hex']}")
    
    # Extract signed transaction hex
    signed_tx_hex2 = signed_tx2["hex"]

except JSONRPCException as e:
    logger.error(f"Error creating or signing transaction: {e}")
    exit(1)

# Step 10: Broadcast the second signed transaction to the network
try:
    # Broadcast the signed transaction
    txid_broadcast2 = rpc_client.sendrawtransaction(signed_tx_hex2)
    logger.info(f"Broadcasted transaction with TXID: {txid_broadcast2}")

except JSONRPCException as e:
    logger.error(f"Error broadcasting transaction: {e}")
    txid_broadcast2 = None  # If broadcasting fails, set txid_broadcast to None

# Step 11: Mine blocks to confirm the second transaction (on regtest)
if txid_broadcast2:
    try:
        logger.info(f"Mining 6 blocks to confirm the transaction {txid_broadcast2}...")
        rpc_client.generatetoaddress(6, address_c)  # Mine 6 blocks to confirm the transaction
        logger.info("6 blocks mined.")
    except JSONRPCException as e:
        logger.error(f"Error mining blocks to confirm the transaction: {e}")
else:
    logger.error("Transaction broadcasting failed. Mining cannot proceed.")

# Step 12: Check balances after the second transaction
try:
    # Check balance of Address C
    balance_c = rpc_client.getreceivedbyaddress(address_c)
    logger.info(f"Balance of Address C after transaction: {balance_c} BTC")

except JSONRPCException as e:
    logger.error(f"Error checking balances: {e}")

# Decode raw transactions for transparency
decoded_tx = rpc_client.decoderawtransaction(raw_tx, True)
decoded_tx2 = rpc_client.decoderawtransaction(raw_tx2, True)

pprint((f"Decoded Transaction 1: {decoded_tx}"))
pprint((f"Decoded Transaction 2: {decoded_tx2}"))


DEBUG:BitcoinRPC:-274-> getnewaddress ["", "legacy"]
DEBUG:BitcoinRPC:<-274- "miuQJtcbYvC7uF12h6ahkjgGhEX4eMsjmx"
DEBUG:BitcoinRPC:-275-> getnewaddress ["", "legacy"]
DEBUG:BitcoinRPC:<-275- "mydo5runmx2GH7kecVweU1jx6AE4cA8Txf"
DEBUG:BitcoinRPC:-276-> getnewaddress ["", "legacy"]
DEBUG:BitcoinRPC:<-276- "mnHTBModaKGUoaKBesyHs3jmWNFk4YxHSq"
INFO:BitcoinRPC:New Legacy address A generated: miuQJtcbYvC7uF12h6ahkjgGhEX4eMsjmx
INFO:BitcoinRPC:New Legacy address B generated: mydo5runmx2GH7kecVweU1jx6AE4cA8Txf
INFO:BitcoinRPC:New Legacy address C generated: mnHTBModaKGUoaKBesyHs3jmWNFk4YxHSq
DEBUG:BitcoinRPC:-277-> sendtoaddress ["miuQJtcbYvC7uF12h6ahkjgGhEX4eMsjmx", 10.0]
DEBUG:BitcoinRPC:<-277- "547968e2b59cc4f1846bfef41b6a27f6ed5ca94dd88111246913207b39f23cf3"
INFO:BitcoinRPC:Sent 10.0 BTC to address A with TXID: 547968e2b59cc4f1846bfef41b6a27f6ed5ca94dd88111246913207b39f23cf3
INFO:BitcoinRPC:Mining 6 blocks to confirm the transaction 547968e2b59cc4f1846bfef41b6a27f6ed5ca94dd88111246913207b3

("Decoded Transaction 1: {'txid': "
 "'9299e18a4bd782ba227a5ce6a0ff8b2c52b800996f4dad33dc2950b05e44a306', 'hash': "
 "'9299e18a4bd782ba227a5ce6a0ff8b2c52b800996f4dad33dc2950b05e44a306', "
 "'version': 2, 'size': 85, 'vsize': 85, 'weight': 340, 'locktime': 0, 'vin': "
 "[{'txid': "
 "'547968e2b59cc4f1846bfef41b6a27f6ed5ca94dd88111246913207b39f23cf3', 'vout': "
 "0, 'scriptSig': {'asm': '', 'hex': ''}, 'sequence': 4294967293}], 'vout': "
 "[{'value': Decimal('9.99990000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_DUP "
 'OP_HASH160 c6bd7ccf3bd0aed2d3ac8b7d62c675ea46dabad7 OP_EQUALVERIFY '
 "OP_CHECKSIG', 'desc': 'addr(mydo5runmx2GH7kecVweU1jx6AE4cA8Txf)#mt6q8dfq', "
 "'hex': '76a914c6bd7ccf3bd0aed2d3ac8b7d62c675ea46dabad788ac', 'address': "
 "'mydo5runmx2GH7kecVweU1jx6AE4cA8Txf', 'type': 'pubkeyhash'}}]}")
("Decoded Transaction 2: {'txid': "
 "'0f49bba2a28b9656d7f830415f0d840f792b7a853fa368618adf733032caa0e8', 'hash': "
 "'0f49bba2a28b9656d7f830415f0d840f792b7a853fa368618adf733032caa0e8', "
 